In [1]:
%load_ext autoreload
%autoreload 2

import os
import copy
import pickle
import warnings

import numpy as np

from astropy.table import Table, QTable, Column

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import lsst.daf.butler as dafButler
import lsst.afw.image as afwImage

# Examples of QA data

- Mainly for Xiaojing and Mingyu

In [4]:
# Merian data repo
root = '/projects/MERIAN/repo'

butler = dafButler.Butler(root)

In [5]:
output_dir = '/tigress/sh19/public_html/cosmos/202203'

## Merian Coadd Catalogs 

### `N708`

- Going to use the "Deep" version since it has pretty uniform coverage

In [6]:
n708_rerun = 'DECam/runs/merian/w_2022_02/t9813_deep_N708'

n708_dataid = dict(tract=9813, band='N708')

n708_coadd = butler.get(
    'objectTable_tract', dataId=n708_dataid, instrument='DECam', skymap='hsc_rings_v1', collections=n708_rerun)

In [7]:
# This is the coadd catalog from the whole Tract, including real and sky objects

len(n708_coadd)

1816938

In [8]:
# The output is a Pandas.DataFrame
type(n708_coadd)

pandas.core.frame.DataFrame

In [9]:
# Unfortunately, the order of the columns is not perfect...you have to search for the one you want
print(n708_coadd.columns.values.tolist())

['shape_xx', 'shape_flag', 'sky_object', 'coord_dec', 'refBand', 'deblend_nChild', 'shape_yy', 'parentObjectId', 'shape_xy', 'merge_peak_sky', 'xErr', 'yErr', 'y', 'detect_isDeblendedSource', 'detect_fromBlend', 'deblend_skipped', 'refFwhm', 'refExtendedness', 'detect_isPrimary', 'footprintArea', 'detect_isPatchInner', 'coord_ra', 'detect_isTractInner', 'xy_flag', 'x', 'detect_isDeblendedModelSource', 'detect_isIsolated', 'skymap', 'tract', 'patch', 'N708_ra', 'N708_decl', 'N708_psfFlux', 'N708_psfFluxErr', 'N708_free_psfFlux', 'N708_free_psfFluxErr', 'N708_free_psfFlux_flag', 'N708_bdE1', 'N708_bdE2', 'N708_bdReB', 'N708_bdReD', 'N708_bdChi2', 'N708_bdFluxB', 'N708_bdFluxBErr', 'N708_bdFluxD', 'N708_bdFluxDErr', 'N708_gaapPsfFlux', 'N708_gaapPsfFluxErr', 'N708_gaap0p5Flux', 'N708_gaap0p5FluxErr', 'N708_gaap0p7Flux', 'N708_gaap0p7FluxErr', 'N708_gaap1p0Flux', 'N708_gaap1p0FluxErr', 'N708_gaap1p5Flux', 'N708_gaap1p5FluxErr', 'N708_gaap2p5Flux', 'N708_gaap2p5FluxErr', 'N708_gaap3p0Flux',

In [12]:
# This flag is for sky object

sky_flag = (n708_coadd['merge_peak_sky'] == True)
sky_flag.sum()

6979

In [19]:
# For real galaxies, we only want the Primary object

prim_flag = (~sky_flag & n708_coadd['detect_isPrimary'] & (n708_coadd['deblend_nChild'] == 0))
prim_flag.sum()

824658

In [21]:
# The DataFrame can be directly saved as Pickle, HDF5, and Parquet format

n708_coadd[sky_flag].to_pickle(
    os.path.join(output_dir, 'coadd', 'n708', 'merian_n708_deep_9813_sky.pkl'))

n708_coadd[prim_flag].to_pickle(
    os.path.join(output_dir, 'coadd', 'n708', 'merian_n708_deep_9813_primary.pkl'))

In [25]:
# If you want to save it as an old-fashioned FITS table, you have to use Astropy.Table

Table.from_pandas(n708_coadd).write(
    os.path.join(output_dir, 'coadd', 'n708', 'merian_n708_deep_9813.fits'), format='fits', overwrite=True)

### `N540`

In [27]:
n540_rerun = 'DECam/runs/merian/w_2022_02/t9813_deep_N540'

n540_dataid = dict(tract=9813, band='N540')

n540_coadd = butler.get(
    'objectTable_tract', dataId=n540_dataid, instrument='DECam', skymap='hsc_rings_v1', collections=n540_rerun)

# Sky objects
sky_flag = (n540_coadd['merge_peak_sky'] == True)
sky_flag.sum()

n540_coadd[sky_flag].to_pickle(
    os.path.join(output_dir, 'coadd', 'n540', 'merian_n540_deep_9813_sky.pkl'))

# Primary objects
prim_flag = (~sky_flag & n540_coadd['detect_isPrimary'] & (n540_coadd['deblend_nChild'] == 0))
prim_flag.sum()

n540_coadd[prim_flag].to_pickle(
    os.path.join(output_dir, 'coadd', 'n540', 'merian_n540_deep_9813_primary.pkl'))

# Full catalog
Table.from_pandas(n540_coadd).write(
    os.path.join(output_dir, 'coadd', 'n540', 'merian_n540_deep_9813.fits'), format='fits', overwrite=True)

## Merian Coadd Image

- Get the coadd image of one `Patch`

### `N708`

In [10]:
# Here I just choose the Patch ID randomly.
n708_img_id = {'tract': 9813, 'patch': 22, 'band': 'N708'}

n708_img = butler.get(
    'deepCoadd', dataId=n708_img_id, instrument='DECam', skymap='hsc_rings_v1', collections=n708_rerun)

In [11]:
# Image dimension
n708_img.getDimensions()

Extent2I(4200, 4200)

In [14]:
# The flux 
flux = n708_img.getImage()

# The object mask
mask = n708_img.getMask()

# The variance 
var = n708_img.getVariance()

In [16]:
# The flux, mask, var are still special objects
# If you need numpy.array, you need to get the array attribution
flux.array

array([[-0.00072371,  0.0358339 ,  0.02275437, ..., -0.03961521,
         0.02224333,  0.04279563],
       [ 0.00036555,  0.00098316,  0.0081486 , ..., -0.01154273,
        -0.00686834,  0.00898801],
       [-0.00409637, -0.03652273, -0.04689546, ...,  0.00605213,
        -0.0313235 ,  0.01018497],
       ...,
       [ 0.01356809,  0.01146072,  0.00304138, ..., -0.06666885,
        -0.06624182, -0.0110684 ],
       [ 0.01405278,  0.03681416,  0.02339144, ..., -0.02573195,
        -0.06419283, -0.00813852],
       [-0.00704356,  0.01506443,  0.01219904, ...,  0.0499026 ,
         0.04059465,  0.01543511]], dtype=float32)

In [15]:
# WCS info
n708_img.getWcs()

FITS standard SkyWcs:
Sky Origin: (150.2479338843, +2.2314049587)
Pixel Origin: (17999, 17999)
Pixel Scale: 0.168 arcsec/pixel

In [17]:
n708_img.getPhotoCalib()

PhotoCalib(spatially constant with mean: 57.544 error: 0)

In [20]:
# HDU 1 is the image
# HDU 2 is the mask
# HDU 3 is the variance

n708_img.writeFits(
    os.path.join(output_dir, 'coadd', 'n708', 'img', 'merian_n708_deep_9813_22_deepcoadd.fits'))

### `N540`

In [23]:
n540_rerun = 'DECam/runs/merian/w_2022_02/t9813_deep_N540'

n540_img_id = {'tract': 9813, 'patch': 22, 'band': 'N540'}

n540_img = butler.get(
    'deepCoadd', dataId=n540_img_id, instrument='DECam', skymap='hsc_rings_v1', collections=n540_rerun)

n540_img.writeFits(
    os.path.join(output_dir, 'coadd', 'n540', 'img', 'merian_n540_deep_9813_22_deepcoadd.fits'))

## Merian Single Exposure Catalogs 

- This is a useful reference: https://pipelines.lsst.io/getting-started/display.html?highlight=calexp

### `N708`

In [24]:
exp_collection = 'DECam/runs/merian/w_2022_02'

In [25]:
# Based on the observational log to get the exposure ID, the filter, and the observing conditions

# Airmass: 1.29
# Seeing: 1.01
# Sky background: 21.09 mag/arcsec^2
# Transparency: 1.0
n708_visit = 971900

In [26]:
n708_src = butler.get(
    "sourceTable_visit", instrument="DECam", visit=n708_visit, collections=exp_collection)

In [27]:
len(n708_src)

204698

In [28]:
# The single exposure catalog doesn't have the filter name in the column name 

print(n708_src.columns.values.tolist())

['coord_ra', 'coord_dec', 'ccdVisitId', 'parentSourceId', 'x', 'y', 'xErr', 'yErr', 'ra', 'decl', 'calibFlux', 'calibFluxErr', 'ap03Flux', 'ap03FluxErr', 'ap03Flux_flag', 'ap06Flux', 'ap06FluxErr', 'ap06Flux_flag', 'ap09Flux', 'ap09FluxErr', 'ap09Flux_flag', 'ap12Flux', 'ap12FluxErr', 'ap12Flux_flag', 'ap17Flux', 'ap17FluxErr', 'ap17Flux_flag', 'ap25Flux', 'ap25FluxErr', 'ap25Flux_flag', 'ap35Flux', 'ap35FluxErr', 'ap35Flux_flag', 'ap50Flux', 'ap50FluxErr', 'ap50Flux_flag', 'ap70Flux', 'ap70FluxErr', 'ap70Flux_flag', 'sky', 'skyErr', 'psfFlux', 'psfFluxErr', 'ixx', 'iyy', 'ixy', 'ixxPSF', 'iyyPSF', 'ixyPSF', 'gaussianFlux', 'gaussianFluxErr', 'extendedness', 'localPhotoCalib', 'localPhotoCalib_flag', 'localPhotoCalibErr', 'localWcs_flag', 'localWcs_CDMatrix_2_1', 'localWcs_CDMatrix_1_1', 'localWcs_CDMatrix_1_2', 'localWcs_CDMatrix_2_2', 'blendedness_abs', 'blendedness_flag', 'blendedness_flag_noCentroid', 'blendedness_flag_noShape', 'apFlux_12_0_flag', 'apFlux_12_0_flag_apertureTruncat

In [36]:
# This catalog includes the sources from all CCDs
# However, the CCD IDs are not present, you have to get it from the ccdVisitId
np.unique(n708_src['ccdVisitId'].array - 97190000)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 62])

In [38]:
# Sky objects
# Notice the column name is different
sky_flag = (n708_src['sky_source'] == True)
print(sky_flag.sum())

n708_src[sky_flag].to_pickle(
    os.path.join(output_dir, 'single', 'n708', 'merian_n708_971900_sky.pkl'))

# Primary objects
prim_flag = (~sky_flag & n708_src['detect_isPrimary'] & (n708_src['deblend_nChild'] == 0))
print(prim_flag.sum())

n708_src[prim_flag].to_pickle(
    os.path.join(output_dir, 'single', 'n708', 'merian_n708_971900_primary.pkl'))

# Full catalog
Table.from_pandas(n708_src).write(
    os.path.join(output_dir, 'single', 'n708', 'merian_n708_971900_src.fits'), format='fits', overwrite=True)

6000
178475


In [43]:
# If you just need the catalog from a single CCD
n708_src_ccd = butler.get(
    "src", instrument="DECam", visit=n708_visit, detector=22, collections=exp_collection)

## Merian Single Exposure Image 

### `N708`

In [45]:
n708_img = butler.get(
    "calexp", instrument="DECam", visit=n708_visit, detector=22, collections=exp_collection)

n708_img.writeFits(
    os.path.join(output_dir, 'single', 'n708', 'img', 'merian_n708_971900_22_img.fits'))

# Very similar to the coadd image

In [46]:
# Get the background subtracted

n708_bkg = butler.get(
    "calexpBackground", instrument="DECam", visit=n708_visit, detector=22, collections=exp_collection)

In [47]:
bkg_img = n708_bkg.getImage()

In [49]:
n708_bkg.writeFits(
    os.path.join(output_dir, 'single', 'n708', 'img', 'merian_n708_971900_22_bkg.fits'))